In [1]:
import pandas as pd
import numpy as np
import networkx as nx

In [135]:
data = pd.read_json('scraping/data.json')
data.loc[data.pref_max_price == '', 'pref_max_price'] = float('inf')
data.pref_max_price = data.pref_max_price.astype(float)

In [136]:
data_full = data.merge(data, how='cross')

In [137]:
data_filtered = data_full[(data_full.offer_price_y<=data_full.pref_max_price_x) & 
                          (data_full.offer_size_y>=data_full.pref_min_size_x) &
                          (data_full.offer_rooms_y<=data_full.pref_min_rooms_x) & 
                          (data_full.ID_x != data_full.ID_y)]

In [183]:
edge_list = []
for i, row in data_filtered.iterrows():
    edge_list.append((row.ID_x, row.ID_y, {'weight':1}))
g = nx.DiGraph()
g.add_edges_from(edge_list)

In [86]:
for i in range(1, 3):
    cycles = list(nx.simple_cycles(g, length_bound=i))
    print('bound', i)
    print(len(cycles))

bound 1
0
bound 2
2965


In [181]:
g_undirected = g.to_undirected(reciprocal=True)
match = nx.max_weight_matching(g_undirected)
res_k2 = []
for u,v in match:
    res_k2.extend([u,v])
res_k2 = set(res_k2)
len(res_k2)

316

In [192]:
list(g.edges)[0]

(164299, 184443)

In [195]:
g[164299][184443]['weight']

1

In [178]:
edge_list = []
for i, row in data_filtered.iterrows():
    #if row.ID_x not in res_k2 and row.ID_y not in res_k2:
    edge_list.append((row.ID_x, row.ID_y))
g2 = nx.DiGraph()
g2.add_edges_from(edge_list)

In [176]:
sorted([2,1,3])

[1, 2, 3]

In [179]:
the_g = g2
for i in range(4, 5):
    print(i)
    while True:
        cycles = list(nx.simple_cycles(the_g, length_bound=i))
        if len(cycles) == 0:
            break
        visited_nodes = set()
        for c in sorted(cycles, key=len, reverse=True):
            if not any([v in visited_nodes for v in c]):
                visited_nodes.update(c)
        print(len(visited_nodes))
        the_g.remove_nodes_from(visited_nodes)
        break

4
332


In [ ]:
for k in range(3, 5):
    print('k', k)
    for v in g2.nodes():
        try:
            c = nx.find_cycle(g2, source=v, orientation='original')
        except nx.NetworkXNoCycle:
            continue
        print(c)
        print(len(c))

In [67]:
for i in range(1, 5):
    cycles = list(nx.simple_cycles(g2, length_bound=i))
    print('bound', i)
    cycle_nodes = set([x for xs in cycles for x in xs])
    print(len(cycle_nodes))

bound 1
0
bound 2
0
bound 3
53
bound 4
123
